In [2]:
import numpy as np
import utils.porter as pt
from Parser import *
from TextRepresenter import *
import collections
from Index import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
Documents = ['the new home has been saled on top forecasts','the home sales rise in july', 'there is an increase in home sales in july',' july encounter a new home sales rise']

In [13]:
file = "../cacmShort-good.txt"
try:
    parser = parse.Parser()
    _ = parser.buildDocCollectionSimple(file)
except FileNotFoundError:
    print("le fichier est introuvable")

indexer = ind.IndexerSimple(parser.collection)
index, inv_index = indexer.get_index()

In [15]:
file = "../data/cacm/cacm.txt"
try:
    parser = parse.Parser()
    _ = parser.buildDocCollectionSimple(file)
except FileNotFoundError:
    print("le fichier est introuvable")

indexer = ind.IndexerSimple(parser.collection)
index, inv_index = indexer.get_index()
print(len(index))

4202


In [17]:
file = "../data/cisi/cisi.txt"
try:
    parser = parse.Parser()
    _ = parser.buildDocCollectionSimple(file,balise='.W')
except FileNotFoundError:
    print("le fichier est introuvable")

indexer = ind.IndexerSimple(parser.collection)
index, inv_index = indexer.get_index()
print(len(index))

2459


In [3]:
Documents = ["the new home has been saled on top forecasts","the home sales rise in july","here is an increase in home sales in july","july encounter a new home sales rise"]

In [6]:
q = "home sales top"

In [7]:
 def booleanScore(q,d):
    stemmer = PorterStemmer()
    dr = stemmer.getTextRepresentation(d)
    qr = stemmer.getTextRepresentation(q)
    for k in qr.keys():
        if k not in dr.keys():
            return 0
    return 1

In [22]:
 def vectorialScore(q,d):
    dr = stemmer.getTextRepresentation(d)
    qr = stemmer.getTextRepresentation(q)
    s = 0
    for k in qr.keys():
        if k in dr.keys():
            s += dr[k]*qr[k]
    return s

In [23]:
booleanScore(q,Documents[0])

1

In [24]:
vectorialScore(q,Documents[0])
dr = stemmer.getTextRepresentation(d)

2

In [27]:
class Weighter:

    def __init__(self,index):
        self.index = index

    def getWeightsForDoc(idDoc):
        pass
    
    def getWeightsForStem(stem):
        pass
    
    def getWeightsForQuery(query):
        pass

In [29]:
class Weighter1(Weighter):
    def __init__(self,index):
        Weighter.__init__(self,index)
        
    def getWeightsForDoc(idDoc):
        return self.index.get_index()[0][idDoc]
    
    def getWeightsForStem(stem):
        return self.index.get_index()[1][stem]
    
    def getWeightsForQuery(query):
        stemmer = PorterStemmer()
        return {w : 1 for w in stemmer.getTextRepresentation(query).keys()}

In [30]:
class Weighter2(Weighter):
    def __init__(self,index):
        Weighter.__init__(self,index)
        
    def getWeightsForDoc(idDoc):
        return self.index.get_index()[0][idDoc]
    
    def getWeightsForStem(stem):
        return self.index.get_index()[1][stem]
    
    def getWeightsForQuery(query):
        stemmer = PorterStemmer()
        return stemmer.getTextRepresentation(query)

In [36]:
class Weighter3(Weighter):
    def __init__(self,index):
        Weighter.__init__(self,index)
        
    def getWeightsForDoc(idDoc):
        return self.index.get_index()[0][idDoc]
    
    def getWeightsForStem(stem):
        return self.index.get_index()[1][stem]
    
    def getWeightsForQuery(query):
        stemmer = PorterStemmer()
        return {w : self.index.idf(w) for w in stemmer.getTextRepresentation(query).keys()}

In [37]:
class Weighter4(Weighter):
    def __init__(self,index):
        Weighter.__init__(self,index)
        
    def getWeightsForDoc(idDoc):
        return [1 + np.log(self.index.get_index()[0][idDoc][t]) for t in self.index.get_index()[0][idDoc]]
    
    def getWeightsForStem(stem):
        return self.index.get_index()[1][stem]
    
    def getWeightsForQuery(query):
        stemmer = PorterStemmer()
        return {w : self.index.idf(w) for w in stemmer.getTextRepresentation(query).keys()}

In [39]:
class Weighter4(Weighter):
    def __init__(self,index):
        Weighter.__init__(self,index)
        
    def getWeightsForDoc(idDoc):
        return [1 + np.log(self.index.get_index()[0][idDoc][t]) * self.index.idf(t)  for t in self.index.get_index()[0][idDoc]]
    
    def getWeightsForStem(stem):
        return self.index.get_index()[1][stem]
    
    def getWeightsForQuery(query):
        stemmer = PorterStemmer()
        return {w : self.index.idf(w) * (1+np.log(stemmer.getTextRepresentation(query)[w])) for w in stemmer.getTextRepresentation(query).keys()}

In [ ]:
 def vectorialCosinusScore(q,d):
    dr = stemmer.getTextRepresentation(d)
    qr = stemmer.getTextRepresentation(q)
    s = 0
    for k in qr.keys():
        if k in dr.keys():
            s += dr[k]*qr[k]
            s1 += dr[k]**2
            s2 += qr[k]**2
    return s/(np.sqrt(s1)+np.sqrt(s2))

In [ ]:
class IRModel:

    def __init__(self,index):
        self.index = index

    def getScores(query):
        pass
    
    def getRanking(query):
        pass
    
class Vectoriel(IRModel):
    def __init__(self,index,weighter,normalized):
        self.index = index
        self.weighter = weighter
        self.normalized = normalized
        
    def getScores(query):
        r = []
        if normalized :
            for d in self.index.get_docs():
                r.append(vectorialCosinusScore(query,self.index.getStrDoc(d)))
        else :
            #produit scalaire
            for d in self.index.get_docs():
                r.append(vectorialScore(query,self.index.getStrDoc(d)))
        return r
        

In [ ]:
class ModeleLangue(IRModel):
    def __init__(self,index,lamb):
        super().__init__(index)
        self.lamb = lamb
    
    def getScores(query):
        r = []
        for d in self.index.get_docs():
            a = 1
            for q in query.keys():
                if q in self.index.get_index[0][d.get_id()].keys():
                    doc = self.index.get_index[0][d.get_id()]
                    a *= (1-self.lamb)* (doc[q]/sum(doc.values()) + self.lamb * self.index.get_index[1][q]
                else :
                                        
                    a *= self.lamb * (sum(self.index.get_index[1][q].values())/
        pt = []
        for q in query.keys():
            r *= (1-self.lamb)*query[]
    